In [1]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict

In [13]:
indir = 'raw_data'
outdir = 'filtered_data'
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
infiles = sorted([os.path.join('raw_data', f) for f in os.listdir('raw_data') 
                      if f.endswith('.txt')])
outfiles = sorted([os.path.join('filtered_data', f) for f in os.listdir('raw_data') 
                       if f.endswith('.txt')])
species = sorted([sp.split('.')[0] for sp in os.listdir('raw_data') 
                      if sp.endswith('.txt')])

In [5]:
os.listdir(indir)

['Miscanthus_floridulus.txt',
 'Miscanthus_sacchariflorus.txt',
 'Miscanthus_transmorrisonensis.txt',
 'Saccharum_spontaneum.txt',
 'Miscanthus_giganteus.txt',
 'Miscanthus_sinensis.txt',
 'Sorghum_bicolor.txt',
 '.ipynb_checkpoints']

In [6]:
def fitler_by_start_end_codon(seq):
    """以ATG起始，并以TAA，TAG或TGA结尾的序列，序列中间没有终止密码子.
    """
    start_codon = 'ATG'
    end_codons = ['TAA', 'TAG', 'TGA']
    seq = [seq[i:i+3] for i in range(0, len(seq), 3)]
    inter_terminal = list(set(end_codons) & set(seq[1:-1]))
    if (seq[0] != start_codon) or (seq[-1] not in end_codons) or (len(inter_terminal)!=0):
        return False
    else:
        return True
    
    
def filter_by_base_num(seq):
    """序列长度为3的倍数.
    """
    if len(seq)%3 != 0:
        return False
    else:
        return True


def filter_by_length(seq):
    """序列长度>=300bp
    """
    if len(seq)<300:
        return False
    else:
        return True

    
    
def read_fasta(infile):
    """读取数据
    """
    cds = defaultdict()
    with open(infile) as f:
        for line in f:
            if line.startswith('>'):
                name = line.strip()
                cds[name] = ''
            else:
                cds[name] += line.strip()
    return cds


def write_fasta(cds, outfile):
    """将过滤得到的序列数据写入文件中,并统计GC含量.
    """
    pos1 = defaultdict(int)
    pos2 = defaultdict(int)
    pos3 = defaultdict(int)
    L_aa = 0
    cds_num_before = len(cds)
    cds_num_after = 0
    with open(outfile, 'wt', encoding='utf-8') as f:
        for name, seq in cds.items():
            if fitler_by_start_end_codon(seq) and filter_by_base_num(seq) and \
                filter_by_length(seq):
                f.write(name+'\n')
                f.write(seq+'\n')
                seq = [seq[i:i+3] for i in range(0, len(seq), 3)]
                L_aa += len(seq)
                cds_num_after += 1
                # 起始密码子和终止密码子不参与GC计算？
                for codon in seq:
                    pos1[codon[0]] += 1
                    pos2[codon[1]] += 1
                    pos3[codon[2]] += 1
    
    GC1 = (pos1['G']+pos1['C'])/(pos1['G']+pos1['C']+pos1['A']+pos1['T'])
    GC2 = (pos2['G']+pos2['C'])/(pos2['G']+pos2['C']+pos2['A']+pos2['T'])
    GC3 = (pos3['G']+pos3['C'])/(pos3['G']+pos3['C']+pos3['A']+pos3['T'])
    
    return GC1, GC2, GC3, L_aa, cds_num_before, cds_num_after
            
            
def run(infile, outfile):
    print('\nProcessing {} ... ...'.format(infile))
    cds = read_fasta(infile)
    GC1, GC2, GC3, L_aa, cds_num_before, cds_num_after = write_fasta(cds, outfile)
    GC123 = (GC1+GC2+GC3)/3
    print('CDS numbers before filter: ', cds_num_before)
    print('CDS numbers after filter: ', cds_num_after)
    print('Total amino acid numbers: ', L_aa)
    print('GC1= {0}, GC2= {1}, GC123= {2}, GC3= {3}'.format(
        round(GC1,3), round(GC2,3), round(GC123,3 ), round(GC3,3)))
    return L_aa, cds_num_before, cds_num_after, \
        round(GC1,3), round(GC2,3), round(GC3,3), round(GC123, 3)

In [8]:
results = {}
for infile, outfile, species in zip(infiles, outfiles, species):
    results[species] = run(infile, outfile)


Processing raw_data/Miscanthus_floridulus.txt ... ...
CDS numbers before filter:  106
CDS numbers after filter:  65
Total amino acid numbers:  19611
GC1= 0.473, GC2= 0.397, GC123= 0.394, GC3= 0.312

Processing raw_data/Miscanthus_sacchariflorus.txt ... ...
CDS numbers before filter:  122
CDS numbers after filter:  64
Total amino acid numbers:  19508
GC1= 0.472, GC2= 0.396, GC123= 0.393, GC3= 0.311

Processing raw_data/Miscanthus_transmorrisonensis.txt ... ...
CDS numbers before filter:  106
CDS numbers after filter:  64
Total amino acid numbers:  19486
GC1= 0.473, GC2= 0.397, GC123= 0.394, GC3= 0.311

Processing raw_data/Saccharum_spontaneum.txt ... ...
CDS numbers before filter:  76
CDS numbers after filter:  48
Total amino acid numbers:  16553
GC1= 0.477, GC2= 0.395, GC123= 0.391, GC3= 0.302

Processing raw_data/Miscanthus_giganteus.txt ... ...
CDS numbers before filter:  106
CDS numbers after filter:  64
Total amino acid numbers:  19469
GC1= 0.474, GC2= 0.397, GC123= 0.394, GC3= 0.

In [9]:
results

{'Miscanthus_floridulus': (19611, 106, 65, 0.473, 0.397, 0.312, 0.394),
 'Miscanthus_sacchariflorus': (19508, 122, 64, 0.472, 0.396, 0.311, 0.393),
 'Miscanthus_transmorrisonensis': (19486, 106, 64, 0.473, 0.397, 0.311, 0.394),
 'Saccharum_spontaneum': (16553, 76, 48, 0.477, 0.395, 0.302, 0.391),
 'Miscanthus_giganteus': (19469, 106, 64, 0.474, 0.397, 0.311, 0.394),
 'Miscanthus_sinensis': (19506, 122, 64, 0.472, 0.396, 0.311, 0.393),
 'Sorghum_bicolor': (17490, 84, 52, 0.476, 0.393, 0.303, 0.39)}

In [10]:
df = pd.DataFrame(results)
df.index = ['L_aa','cds_num_before_filter','cds_num_after_filter','GC1','GC2','GC3','GC123']
columns = sorted(df.columns)
df = df[columns]
df.columns = [' '.join(c.split('_')) for c in df.columns]
df

,Miscanthus floridulus,Miscanthus giganteus,Miscanthus sacchariflorus,Miscanthus sinensis,Miscanthus transmorrisonensis,Saccharum spontaneum,Sorghum bicolor
L_aa,19611.000,19469.000,19508.000,19506.000,19486.000,16553.000,17490.000
cds_num_before_filter,106.000,106.000,122.000,122.000,106.000,76.000,84.000
cds_num_after_filter,65.000,64.000,64.000,64.000,64.000,48.000,52.000
GC1,0.473,0.474,0.472,0.472,0.473,0.477,0.476
GC2,0.397,0.397,0.396,0.396,0.397,0.395,0.393
GC3,0.312,0.311,0.311,0.311,0.311,0.302,0.303
GC123,0.394,0.394,0.393,0.393,0.394,0.391,0.390


In [12]:
df.to_excel('step1_results_GC.xlsx', header=True, index=True, encoding='utf-8')

In [117]:
pwd

'/data/shexuan/sjj/new_paper'

In [135]:
wd = '/data/shexuan/sjj/new_paper/filtered_data/'
fastas = [os.path.join(wd, f) for f in os.listdir('filtered_data') if f.endswith('txt')]
for i in fastas:
    print(i)

/data/shexuan/sjj/new_paper/filtered_data/Miscanthus_floridulus.txt
/data/shexuan/sjj/new_paper/filtered_data/Miscanthus_sacchariflorus.txt
/data/shexuan/sjj/new_paper/filtered_data/Miscanthus_transmorrisonensis.txt
/data/shexuan/sjj/new_paper/filtered_data/Saccharum_spontaneum.txt
/data/shexuan/sjj/new_paper/filtered_data/Miscanthus_giganteus.txt
/data/shexuan/sjj/new_paper/filtered_data/Miscanthus_sinensis.txt
/data/shexuan/sjj/new_paper/filtered_data/Sorghum_bicolor.txt


In [ ]:
species_dict = defaultdict()

for fasta in fastas:
    species = fasta.split('/')[-1].split('.')[0]
    with open(fasta) as f:
        for line in f:
            if line.startswith('>'):
                pass
            else:
                species_dict[species] += line.strip()